# TLDR

The contact distribution from positive cases is a biased representation of the underlying contact distribution of the population. (People with more contacts are more likely to become infected and contribute to the observed contact distribution, and vice versa.)

We would like to use Bayesian inference to obtain a more accurate underlying contact distribution. 

In [29]:
%load_ext autoreload
%autoreload 2

import numpy as np

import pyro
import pyro.distributions as dist
from pyro.infer import MCMC, NUTS, SVI, Trace_ELBO, Predictive

import torch
import pandas as pd
import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
def observe_contact_distribution(N: int, lamda: float, beta: float, data: torch.Tensor =None):

    r""" 
    Args:
        N: population size
        lamda: prevalence
        beta: probability of infection given contact with positive
    """

    # p = distribution of the number of contacts of a person; use Dirichlet prior
    p = pyro.sample(
        'p', 
        dist.Dirichlet(concentration=torch.tensor([0.1, 0.1, 0.1])), 
    )
    binomial_counts = pyro.deterministic("binomial_counts", (N*p).int())
    
    # print('p', p)
    # print('binomial_counts', binomial_counts)

    p_inf = 1-(1-lamda*beta)**torch.arange(3)

    with pyro.plate('data'):
        obs = pyro.sample(
            'obs', 
            dist.Binomial(
                total_count=binomial_counts, # this throws an error
                # total_count=20, # this runs
                probs=p_inf),
            obs=data 
        )
    return obs

In [31]:
N=200 # population size
lamda = 0.1 # prevalence
beta=0.1 # probability of infection given contact with positive person
y = torch.tensor([5,5,10]) # observed contact distribution, i.e., 5 have 0, 5 have 1, 10 have 2 contacts
# y = torch.tensor([5])

nuts_kernel = NUTS(observe_contact_distribution) # also called "sampler"
mcmc = MCMC(nuts_kernel, warmup_steps=5, num_samples=10)


In [32]:
start_time = time.time()
mcmc.run(N, lamda, beta, y)
print(f"Took {time.time()-start_time} seconds")

Sample: 100%|██████████| 15/15 [00:01, 12.16it/s, step size=1.53e-01, acc. prob=0.306]

Took 1.2381768226623535 seconds


In [33]:
hmc_samples = {k: v.detach().cpu().numpy() for k, v in mcmc.get_samples().items()}


In [34]:
# Utility function to print latent sites' quantile information.
def summary(samples):
    site_stats = {}
    for site_name, values in samples.items():
        marginal_site = pd.DataFrame(values)
        describe = marginal_site.describe(percentiles=[.05, 0.25, 0.5, 0.75, 0.95]).transpose()
        site_stats[site_name] = describe[["mean", "std", "5%", "25%", "50%", "75%", "95%"]]
    return site_stats

In [35]:
for site, values in summary(hmc_samples).items():
    print("Site: {}".format(site))
    print(values, "\n")

Site: p
       mean       std        5%       25%       50%       75%       95%
0  0.290979  0.101167  0.147689  0.206078  0.306563  0.381423  0.402956
1  0.158898  0.084343  0.080959  0.088198  0.148454  0.197475  0.286960
2  0.550123  0.086977  0.440032  0.507497  0.519916  0.635689  0.654871 

